# 自定义代理程序与工具检索

这个笔记本中引入的新颖想法是利用检索来选择要用于回答代理程序查询的工具集。当你有许多工具可供选择时，这是很有用的。你无法在提示中放入所有工具的描述（因为上下文长度的问题），所以你可以在运行时动态地选择你想要考虑使用的 N 个工具。

在这个笔记本中，我们将创建一个有些牵强的例子。我们将有一个合法的工具（搜索），然后有 99 个假的工具，它们只是胡言乱语。然后我们将在提示模板中添加一个步骤，该步骤接受用户输入并检索与查询相关的工具。

## 设置环境

进行必要的导入等操作。

In [1]:
import re  # 导入正则表达式模块
from typing import Union  # 导入Union类型

from langchain.agents import (  # 导入langchain.agents模块中的以下类
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
    Tool,
)
from langchain.chains import LLMChain  # 导入langchain.chains模块中的LLMChain类
from langchain.prompts import StringPromptTemplate  # 导入langchain.prompts模块中的StringPromptTemplate类
from langchain_community.utilities import SerpAPIWrapper  # 导入langchain_community.utilities模块中的SerpAPIWrapper类
from langchain_core.agents import AgentAction, AgentFinish  # 导入langchain_core.agents模块中的AgentAction和AgentFinish类
from langchain_openai import OpenAI  # 导入langchain_openai模块中的OpenAI类

## 设置工具

我们将创建一个合法的工具（搜索）和99个假工具。

In [12]:
# 定义代理可以使用的工具来回答用户查询
search = SerpAPIWrapper()  # 创建一个SerpAPIWrapper对象
search_tool = Tool(  # 创建一个名为Search的工具对象
    name="Search",  # 工具名称为Search
    func=search.run,  # 使用search对象的run方法作为工具的功能
    description="useful for when you need to answer questions about current events",  # 描述工具的用途
)


def fake_func(inp: str) -> str:  # 定义一个名为fake_func的函数，接受一个字符串类型的参数，返回一个字符串类型的结果
    return "foo"  # 返回字符串"foo"


fake_tools = [  # 创建一个包含多个工具对象的列表
    Tool(  # 创建一个名为foo-i的工具对象
        name=f"foo-{i}",  # 工具名称为"foo-i"
        func=fake_func,  # 使用fake_func函数作为工具的功能
        description=f"a silly function that you can use to get more information about the number {i}",  # 描述工具的用途
    )
    for i in range(99)  # 循环创建99个工具对象
]
ALL_TOOLS = [search_tool] + fake_tools  # 将search_tool和fake_tools合并为一个包含所有工具的列表

## 工具检索器

我们将使用向量存储来为每个工具描述创建嵌入。然后，对于传入的查询，我们可以为该查询创建嵌入，并对相关工具进行相似性搜索。

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [5]:
# 创建一个包含Document对象的列表，每个Document对象包含一个页面内容和元数据
docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(ALL_TOOLS)
]

In [6]:
# 使用FAISS库从文档中构建向量存储，使用OpenAIEmbeddings进行嵌入
vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [18]:
# 创建一个检索器对象
retriever = vector_store.as_retriever()

def get_tools(query):
    # 使用检索器对象检索相关文档
    docs = retriever.invoke(query)
    # 返回检索到的文档对应的工具列表
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

我们现在可以测试这个检索器，看看它是否能正常工作。

In [19]:
get_tools("whats the weather?")

[Tool(name='Search', description='useful for when you need to answer questions about current events', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x114b28a90>, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='', aiosession=None)>, coroutine=None),
 Tool(name='foo-95', description='a silly function that you can use to get more information about the number 95', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x114b28a90>, func=<function fake_func at 0x15e5bd1f0>, coroutine=None),
 Tool(name='foo-12', description='a silly function that you can use to get more information about the number 12', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManage

In [20]:
# 调用函数并传入参数
get_tools("whats the number 13?")

[Tool(name='foo-13', description='a silly function that you can use to get more information about the number 13', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x114b28a90>, func=<function fake_func at 0x15e5bd1f0>, coroutine=None),
 Tool(name='foo-12', description='a silly function that you can use to get more information about the number 12', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x114b28a90>, func=<function fake_func at 0x15e5bd1f0>, coroutine=None),
 Tool(name='foo-14', description='a silly function that you can use to get more information about the number 14', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x114b28a90>, func=<function fake_func at 0x15e5bd1f0>, coroutine=None),
 Tool(name='foo-11', description='a silly function that you can use to get more information about th

## 提示模板

提示模板非常标准，因为我们实际上并没有在实际提示模板中改变太多逻辑，而是改变了检索的方式。

In [21]:
# 设置基本模板
template = """以海盗的方式回答以下问题，尽力而为。你可以使用以下工具：

{tools}

使用以下格式：

问题：你必须回答的输入问题
思考：你应该始终考虑该做什么
行动：要采取的行动，应为[{tool_names}]之一
行动输入：行动的输入
观察：行动的结果
...（这个思考/行动/行动输入/观察可以重复N次）
思考：我现在知道最终答案了
最终答案：原始输入问题的最终答案

开始！在给出最终答案时，请以海盗的方式说话。使用大量的“Arg”

问题：{input}
{agent_scratchpad}"""

自定义提示模板现在具有`tools_getter`的概念，我们在输入上调用它来选择要使用的工具。

In [52]:
from typing import Callable

# 设置一个提示模板
class CustomPromptTemplate(StringPromptTemplate):
    # 要使用的模板
    template: str
    ############## 新增 ######################
    # 可用工具的列表
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # 获取中间步骤（AgentAction，Observation元组）
        # 以特定方式格式化它们
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # 将agent_scratchpad变量设置为该值
        kwargs["agent_scratchpad"] = thoughts
        ############## 新增 ######################
        tools = self.tools_getter(kwargs["input"])
        # 从提供的工具列表创建一个tools变量
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # 创建一个工具名称列表，用于提供的工具
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [53]:
prompt = CustomPromptTemplate(
    template=template,  # 使用给定的模板创建一个CustomPromptTemplate对象
    tools_getter=get_tools,  # 使用get_tools函数获取工具列表
    # 这里省略了agent_scratchpad、tools和tool_names变量，因为它们是动态生成的
    # 这里包括intermediate_steps变量，因为它是必需的
    input_variables=["input", "intermediate_steps"],  # 设置输入变量为["input", "intermediate_steps"]
)

## 输出解析器

输出解析器与之前的笔记本保持不变，因为我们不会改变任何关于输出格式的内容。

In [54]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否应该结束
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # 返回值通常是一个只有一个`output`键的字典
                # 目前不建议尝试其他操作 :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # 解析动作和动作输入
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"无法解析LLM输出: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # 返回动作和动作输入
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [55]:
output_parser = CustomOutputParser()  # 创建一个名为output_parser的对象，类型为CustomOutputParser()，用于解析输出结果。

## 设置LLM、停止序列和代理

与之前的笔记本相同。

In [56]:
# 导入OpenAI类
from openai import OpenAI

# 创建OpenAI实例，并设置温度参数为0
llm = OpenAI(temperature=0)

In [57]:
# 创建一个LLM链，包括LLM和一个提示
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [58]:
# 导入所需的模块
from typing import List
from llm import LLMSingleActionAgent, get_tools

# 获取工具列表
tools = get_tools("whats the weather?")

# 提取工具名称
tool_names = [tool.name for tool in tools]

# 创建LLMSingleActionAgent对象
# 参数说明：
# - llm_chain: LLM链
# - output_parser: 输出解析器
# - stop: 停止标志
# - allowed_tools: 允许使用的工具名称列表
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)

## 使用代理

现在我们可以使用它了！

In [59]:
# 创建一个AgentExecutor对象，使用Agent和tools作为参数，并设置verbose为True
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [60]:
# 调用agent_executor模块的run函数，并传入参数"What's the weather in SF?"，即询问天气的问题
agent_executor.run("What's the weather in SF?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the weather is in SF
Action: Search
Action Input: Weather in SF

Observation:Mostly cloudy skies early, then partly cloudy in the afternoon. High near 60F. ENE winds shifting to W at 10 to 15 mph. Humidity71%. UV Index6 of 10. I now know the final answer
Final Answer: 'Arg, 'tis mostly cloudy skies early, then partly cloudy in the afternoon. High near 60F. ENE winds shiftin' to W at 10 to 15 mph. Humidity71%. UV Index6 of 10.

> Finished chain.


"'Arg, 'tis mostly cloudy skies early, then partly cloudy in the afternoon. High near 60F. ENE winds shiftin' to W at 10 to 15 mph. Humidity71%. UV Index6 of 10."